# Edge Cloud Joint Inference with Seldon Core and Tempo

Description

## Setup Environment

In [7]:
!conda env create --name edge-cloud-inference --file ./conda/edge-cloud-inference.yaml

In [6]:
!conda activate edge-cloud-inference

## Train models 

In [ ]:
!pip install git+https://github.com/SachinVarghese/tempo.git@tempo-k8s-nodename#egg=mlops-tempo&subdirectory=tempo

In [2]:
import os
from tempo.utils import logger
import logging
logger.setLevel(logging.ERROR)
logging.basicConfig(level=logging.ERROR)
ARTIFACTS_FOLDER = os.getcwd()+"/artifacts"

In [2]:
# %load src/train.py
from src.data import IrisData
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import joblib

EdgeModelFolder = "edge"
CloudModelFolder = "cloud"


def train_edge_model(data: IrisData, artifacts_folder: str):
    logreg = LogisticRegression(C=1e5)
    logreg.fit(data.X, data.y)
    with open(f"{artifacts_folder}/{EdgeModelFolder}/model.joblib", "wb") as f:
        joblib.dump(logreg, f)


def train_cloud_model(data: IrisData, artifacts_folder: str):
    clf = XGBClassifier()
    clf.fit(data.X, data.y)
    clf.save_model(f"{artifacts_folder}/{CloudModelFolder}/model.bst")


In [3]:
from src.data import IrisData
from src.train import train_edge_model, train_cloud_model
data = IrisData()
train_edge_model(data, ARTIFACTS_FOLDER)
train_cloud_model(data, ARTIFACTS_FOLDER)


/home/sachin/miniconda3/envs/edge-cloud-inference/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:59:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Create Tempo artifacts

In [4]:
# %load src/tempo.py
from typing import Tuple

import numpy as np
from src.train import CloudModelFolder, EdgeModelFolder

from tempo.serve.metadata import ModelFramework, RuntimeOptions, KubernetesOptions
from tempo.seldon.k8s import SeldonCoreOptions
from tempo.serve.model import Model
from tempo.serve.pipeline import Pipeline, PipelineModels
from tempo.serve.utils import pipeline

PipelineFolder = "joint-classifier"
EdgePredictionTag = "edge prediction"
CloudPredictionTag = "cloud prediction"

edgeRuntimeOptions = RuntimeOptions()  # Docker Runtime
edgeRuntimeOptions.k8s_options = KubernetesOptions(
    replicas=1, namespace="production", nodeName="edge-compute",
)


cloudRuntimeOptions = SeldonCoreOptions()  # Kubernetes Runtime
cloudRuntimeOptions.k8s_options = KubernetesOptions(
    replicas=2,
    namespace="production",
    nodeName="gke-kubeedge-cloudcore-default-pool-4dbe91a1-v7e5",
)


def get_tempo_artifacts(artifacts_folder: str) -> Tuple[Pipeline, Model, Model]:

    cloud_model = Model(
        name="cloud-model",
        platform=ModelFramework.XGBoost,
        local_folder=f"{artifacts_folder}/{CloudModelFolder}",
        uri="s3://tempo/joint-inference/cloud",
        description="An Cloud based Iris classification model",
        runtime_options=cloudRuntimeOptions,
    )

    edge_model = Model(
        name="edge-model",
        platform=ModelFramework.SKLearn,
        local_folder=f"{artifacts_folder}/{EdgeModelFolder}",
        uri="s3://tempo/joint-inference/edge",
        description="An Edge based Iris classification model",
        runtime_options=edgeRuntimeOptions,
    )

    @pipeline(
        name="joint-classifier",
        uri="s3://tempo/basic/pipeline",
        local_folder=f"{artifacts_folder}/{PipelineFolder}",
        models=PipelineModels(edge_inference=edge_model, cloud_inference=cloud_model),
        description="A pipeline to make an edge based prediction or cloud based joint prediction for Iris classification",
        runtime_options=edgeRuntimeOptions,
    )
    def classifier(payload: np.ndarray) -> Tuple[np.ndarray, str]:
        # Custom Logic for hard example mining based on threshold, IBT, Cross Entropy etc
        res1 = classifier.models.edge_inference(input=payload)
        if res1[0] == 1:
            return res1, EdgePredictionTag
        else:
            return classifier.models.cloud_inference(input=payload), CloudPredictionTag

    return classifier, edge_model, cloud_model


In [3]:
from src.tempo import get_tempo_artifacts
classifier, edge_model, cloud_model = get_tempo_artifacts(ARTIFACTS_FOLDER)

# Save Classifier

In [7]:
from tempo.serve.loader import save
save(classifier)

Packing environment at '/home/sachin/miniconda3/envs/tempo-ccf3b3d9-fe97-4bc0-96e8-ed2ea66cfcbf' to '/home/sachin/projects/mlops/edge-cloud-inference/artifacts/joint-classifier/environment.tar.gz'
[########################################] | 100% Completed | 19.5s


# Deploy to Docker

In [32]:
from tempo.serve.metadata import RuntimeOptions
from tempo import deploy
runtime_options = RuntimeOptions()
remote_model = deploy(classifier, options=runtime_options)

## Make predictions 

### Edge

In [33]:
!docker run --rm --network=tempo curlimages/curl:7.78.0 -XPOST http://172.18.0.2:9000/v2/models/edge-model/infer -s -H "Content-Type: application/json" -d '{"inputs": [{ "name":"dimensions", "data": [0.3,0.6,4.2,3.1], "datatype": "FP64", "shape":[1,4] }]}'

{"model_name":"edge-model","model_version":null,"id":"de4599d3-eee1-4a88-ab87-1b4a84a7262b","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"FP32","parameters":null,"data":[2]}]}

### Cloud

In [34]:
!docker run --rm --network=tempo curlimages/curl:7.78.0 -XPOST http://172.18.0.3:9000/v2/models/cloud-model/infer -s -H "Content-Type: application/json" -d '{"inputs": [{ "name":"dimensions", "data": [0.3,0.6,4.2,3.1], "datatype": "FP64", "shape":[1,4] }]}'

{"model_name":"cloud-model","model_version":null,"id":"1a2e0dcd-6f64-4c15-b65b-cf6ed58453b8","parameters":null,"outputs":[{"name":"predict","shape":[1,3],"datatype":"FP32","parameters":null,"data":[[0.007310844957828522,0.031725041568279266,0.9609640836715698]]}]}

### Joint Inference

In [35]:
!docker run --rm --network=tempo curlimages/curl:7.78.0 -XPOST http://172.18.0.4:9000/v2/models/joint-classifier/infer -s -H "Content-Type: application/json" -d '{"inputs": [{ "name":"dimensions", "data": [0.3,0.6,4.2,3.1], "datatype": "FP64", "shape":[1,4] }]}'

{"model_name":"joint-classifier","model_version":null,"id":"77c13e4b-e184-4da6-b684-a89eca32ee00","parameters":null,"outputs":[{"name":"output0","shape":[1,3],"datatype":"FP32","parameters":null,"data":[0.007310844957828522,0.031725041568279266,0.9609640836715698]},{"name":"output1","shape":[16],"datatype":"BYTES","parameters":null,"data":[99,108,111,117,100,32,112,114,101,100,105,99,116,105,111,110]}]}

# Deploy to Kubernetes

In [15]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/setup/storageInit/rclone.conf")

In [16]:
from tempo.serve.loader import upload
upload(edge_model)
upload(cloud_model)
upload(classifier)

In [37]:
!kubectl create ns production
!kubectl apply -f src/rbac -n production

Error from server (AlreadyExists): namespaces "production" already exists
secret/minio-secret configured
serviceaccount/tempo-pipeline unchanged
role.rbac.authorization.k8s.io/tempo-pipeline unchanged
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding unchanged


In [4]:
from tempo.seldon.k8s import SeldonCoreOptions
runtime_options = SeldonCoreOptions()

In [5]:
from tempo import deploy
remote_model = deploy(classifier, options=runtime_options)

In [57]:
from tempo.seldon.k8s import SeldonKubernetesRuntime
k8s_runtime = SeldonKubernetesRuntime(runtime_options)
models = k8s_runtime.list_models(namespace="production")
print("Name\t\tDescription")
for model in models:
    details = model.get_tempo().model_spec.model_details
    print(f"{details.name}\t{details.description}")

Name		Description
cloud-model	An Cloud based Iris classification model
edge-model	An Edge based Iris classification model
joint-classifier	A pipeline to make an edge based prediction or cloud based joint prediction for Iris classification


## Make predictions

### Edge

In [58]:
!docker run --rm curlimages/curl:7.78.0 -XPOST http://172.17.0.2:9000/v2/models/edge-model/infer -s -H "Content-Type: application/json" -d '{"inputs": [{ "name":"dimensions", "data": [0.3,0.6,4.2,3.1], "datatype": "FP64", "shape":[1,4] }]}'

{"model_name":"edge-model","model_version":"v1","id":"8cb79dfc-68ca-40e0-9fc9-d494e7b4a8f5","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"FP32","parameters":null,"data":[2]}]}

### Cloud

In [64]:
import numpy as np
models[0].predict(np.array([[0.3,0.6,4.2,3.1]])) # via cloud ingress

array([[0.00731084, 0.03172504, 0.9609641 ]], dtype=float32)

### Joint Inference

In [61]:
remote_model.predict(np.array([[0.3,0.6,4.2,3.1]])) # Will fail
# models[1].predict(np.array([[0.3,0.6,4.2,3.1]]))  # Will fail
# models[2].predict(np.array([[0.3,0.6,4.2,3.1]]))  # Will fail

In [66]:
!docker run --rm curlimages/curl:7.78.0 -XPOST http://172.17.0.3:9000/v2/models/joint-classifier/infer -s -H "Content-Type: application/json" -d '{"inputs": [{ "name":"dimensions", "data": [0.3,0.6,4.2,3.1], "datatype": "FP64", "shape":[1,4] }]}'

{"model_name":"joint-classifier","model_version":null,"id":"9c756d4e-959a-4ce7-b495-8d3404261b1f","parameters":null,"outputs":[{"name":"output0","shape":[1,3],"datatype":"FP32","parameters":null,"data":[0.007310844957828522,0.031725041568279266,0.9609640836715698]},{"name":"output1","shape":[16],"datatype":"BYTES","parameters":null,"data":[99,108,111,117,100,32,112,114,101,100,105,99,116,105,111,110]}]}

## Clean up

In [6]:
remote_model.undeploy()

In [ ]:
!kubectl delete ns production